In [3]:
# Did not use this model -- did not produce promising results. Scrapped pretty early on.

In [1]:
! gcsfuse --implicit-dirs --rename-dir-limit=100  --max-conns-per-host=100 hms_applied_cv "bucket"

{"time":"05/03/2024 01:32:11.500110","severity":"INFO","msg":"Start gcsfuse/1.4.0 (Go version go1.21.5) for app \"\" using mount point: /home/jupyter/bucket\n"}


In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from aeon.classification.hybrid import HIVECOTEV2
from sklearn.preprocessing import 
from sklearn.metrics import log_loss
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import joblib
import time

In [3]:
def load_parquet_sample(file_path, n_samples=1):
    df = pd.read_parquet(file_path)
    return df.sample(n=n_samples)

def load_eeg_sample(row):
    eeg_file_path = f'bucket/hms-harmful-brain-activity-classification/train_eegs/{row.eeg_id}.parquet'
    return load_parquet_sample(eeg_file_path)

In [4]:
# Load metadata
print('Loading metadata...')
metadata = pd.read_csv('bucket/hms-harmful-brain-activity-classification/cleaned_train.csv')

metadata_filtered = metadata[metadata['is_center'] == True]

metadata_sample = metadata_filtered.sample(frac=.3, random_state=2)
print('Metadata loaded.')

# eeg_data_samples = []
# spectrogram_samples = []
# total_samples = len(metadata_sample)
# processed_samples = 0  

# print("Starting to load EEG samples...")
# for idx, row in metadata_sample.iterrows():
#     processed_samples += 1
#     if idx % 100 == 0:  # Print progress every 10 samples
#         print(f"Progress: {processed_samples}/{total_samples} samples loaded")
#     eeg_sample = load_parquet_sample(f'bucket/hms-harmful-brain-activity-classification/train_eegs/{row["eeg_id"]}.parquet')
#     eeg_data_samples.append(eeg_sample)
#     # spectrogram_sample = load_parquet_sample(f'bucket/hms-harmful-brain-activity-classification/train_spectrograms/{row["spectrogram_id"]}.parquet')
#     # spectrogram_samples.append(spectrogram_sample)

eeg_data_samples = []
total_samples = len(metadata_sample)
print("Starting to load EEG samples in parallel...")

with ProcessPoolExecutor() as executor:
    # submit tasks to load EEG samples in parallel
    futures = [executor.submit(load_eeg_sample, row) for index, row in metadata_sample.iterrows()]

    # collect results as they are completed
    for i, future in enumerate(futures):
        eeg_data_samples.append(future.result())
        if (i + 1) % 100 == 0:  # Print progress every 10 samples
            print(f"Progress: {i + 1}/{total_samples} samples loaded")

            
eeg_data = pd.concat(eeg_data_samples, ignore_index=True)

# drop any missing values
indices_kept = eeg_data.dropna().index
eeg_data = eeg_data.dropna()
metadata_sample_filtered = metadata_sample.iloc[indices_kept]

# spectrogram_data = pd.concat(spectrogram_samples, ignore_index=True)
print(eeg_data.shape)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(eeg_data, metadata_sample_filtered['expert_consensus'], test_size=0.2, random_state=42)

# train model
print("Starting model training...")
start_time = time.time()
model = HIVECOTEV2(time_limit_in_minutes=0.2)
model.fit(X_train, y_train)
end_time = time.time()
print(f"Model training completed in {end_time - start_time:.2f} seconds.")


# save model, to use model: loaded_model = joblib.load(model_filename)
model_filename = 'hivecotev2_model.joblib'
joblib.dump(model, model_filename)

# predict
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test) # get probability distribution

# evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

lb = LabelBinarizer()
y_test_one_hot = lb.fit_transform(y_test)
loss = log_loss(y_test_one_hot, y_pred_proba)
print(f'loss: {loss}')

Loading metadata...
Metadata loaded.
Starting to load EEG samples in parallel...
Progress: 100/6447 samples loaded
Progress: 200/6447 samples loaded
Progress: 300/6447 samples loaded
Progress: 400/6447 samples loaded
Progress: 500/6447 samples loaded
Progress: 600/6447 samples loaded
Progress: 700/6447 samples loaded
Progress: 800/6447 samples loaded
Progress: 900/6447 samples loaded
Progress: 1000/6447 samples loaded
Progress: 1100/6447 samples loaded
Progress: 1200/6447 samples loaded
Progress: 1300/6447 samples loaded
Progress: 1400/6447 samples loaded
Progress: 1500/6447 samples loaded
Progress: 1600/6447 samples loaded
Progress: 1700/6447 samples loaded
Progress: 1800/6447 samples loaded
Progress: 1900/6447 samples loaded
Progress: 2000/6447 samples loaded
Progress: 2100/6447 samples loaded
Progress: 2200/6447 samples loaded
Progress: 2300/6447 samples loaded
Progress: 2400/6447 samples loaded
Progress: 2500/6447 samples loaded
Progress: 2600/6447 samples loaded
Progress: 2700/644

In [5]:
def predict_eeg(model, file_path):
    eeg_data = load_parquet_sample(file_path)
    predictions = model.predict_proba(eeg_data)
    return predictions.flatten()

loaded_model = joblib.load('hivecotev2_model.joblib')
test_eegs_dir = Path('bucket/hms-harmful-brain-activity-classification/test_eegs')
predictions_list = []
for file in test_eegs_dir.iterdir():
    if file.suffix == '.parquet':
        # get eeg id file name
        eeg_id = file.stem
        
        # Predict the probabilities for the current EEG file
        probabilities = predict_eeg(loaded_model, file)
        
        # Format the prediction record and append to the list
        predictions_list.append([eeg_id] + list(probabilities))
        
predictions_df = pd.DataFrame(predictions_list, columns=['eeg_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])
predictions_df.to_csv('submission.csv', index=False)
print(f'Submissionn shape: {predictions_df.shape}')

Submissionn shape: (1, 7)


In [74]:
eeg_test = load_parquet_sample(f'bucket/hms-harmful-brain-activity-classification/test_eegs/3911565283.parquet') # our submission is only going to be one line?
y_test_proba = model.predict_proba(eeg_test)
print(y_test_proba)


[[0.         0.19952867 0.         0.         0.53939523 0.26107611]]


In [11]:
import sklearn

print(f"Scikit-learn version: {sklearn.__version__}")

Scikit-learn version: 1.3.2


In [1]:
!pip download scikit-learn==1.3.2 -d ""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 42.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 35.2 MB/s eta 0:00:0000:0100:01
Saved ./scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./joblib-1.3.2-py3-none-any.whl
Saved ./numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./threadpoolctl-3.3.0-py3-none-any.whl
Successfully downloaded scikit-learn joblib numpy scipy threadpoolctl


In [ ]:
# use in kaggle for scikit-learn version
!pip install -U scikit-learn==1.3.2
!pip freeze | grep scikit-learn